In [1]:
import os
import cv2 # pip install opencv-python
import  pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras 
from glob import glob
from keras.models import Sequential
from keras.layers import BatchNormalization,MaxPool2D,Dense,Conv2D,Flatten
from keras.callbacks import EarlyStopping,LearningRateScheduler
from sklearn.svm import SVC
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from tqdm import tqdm
from PIL import Image
warnings.filterwarnings('ignore')

2022-11-17 05:51:50.792443: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 05:51:50.971680: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-17 05:51:50.983191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-17 05:51:50.983228: I tensorflo

**1. Import the data**

In [2]:
vehicle_dir = 'data/vehicles/'
non_vehicle_dir = 'data/non-vehicles/'

In [3]:
vehicle_names = os.listdir(vehicle_dir)
non_vehicle_names = os.listdir(non_vehicle_dir)

**2. Explore the data**

In [4]:
print("No of vehicle images:", len(vehicle_names))
print("No of non-vehicle images:", len(non_vehicle_names))

No of vehicle images: 8792
No of non-vehicle images: 8968


**3. Prepare the Data**

In [5]:
cat = {
    'non-vehicles': 0,
    'vehicles': 1
}

# get all the paths for vehicles and non-vehicles
def get_paths(folder):
    
    path = []
    labels = []
    for dirname, _, filenames in os.walk(folder):
        dir_class = dirname.split('/')[1] # either vehicles or non-vehicles
        for filename in filenames:
            path.append(os.path.join(dirname, filename))
            labels.append(cat[dir_class])
    
    return pd.DataFrame({'path': path, 'class': labels})

In [6]:
vehicles_df = get_paths(vehicle_dir)
vehicles_df.head()

path  class
0  data/vehicles/left (739).png      1
1   data/vehicles/far (366).png      1
2   data/vehicles/far (644).png      1
3        data/vehicles/1884.png      1
4        data/vehicles/5462.png      1

In [7]:
non_vehicles_df = get_paths(non_vehicle_dir)
non_vehicles_df.head()

path  class
0  data/non-vehicles/extra2371.png      0
1  data/non-vehicles/image2890.png      0
2   data/non-vehicles/extra643.png      0
3  data/non-vehicles/image3162.png      0
4  data/non-vehicles/image3303.png      0

In [8]:
df = pd.concat([vehicles_df, non_vehicles_df])
df.shape

(17760, 2)

In [9]:
df.head()

path  class
0  data/vehicles/left (739).png      1
1   data/vehicles/far (366).png      1
2   data/vehicles/far (644).png      1
3        data/vehicles/1884.png      1
4        data/vehicles/5462.png      1

In [10]:
# shuffle dataset using sample
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
df.head()

path  class
0           data/vehicles/5854.png      1
1  data/non-vehicles/extra2563.png      0
2      data/vehicles/far (236).png      1
3  data/non-vehicles/image3515.png      0
4           data/vehicles/1632.png      1

In [11]:
# sample for 200 rows first
sample_df = df.iloc[:200, :]
sample_df

path  class
0             data/vehicles/5854.png      1
1    data/non-vehicles/extra2563.png      0
2        data/vehicles/far (236).png      1
3    data/non-vehicles/image3515.png      0
4             data/vehicles/1632.png      1
..                               ...    ...
195            data/vehicles/684.png      1
196  data/non-vehicles/extra2991.png      0
197  data/non-vehicles/image2464.png      0
198           data/vehicles/1394.png      1
199      data/vehicles/far (830).png      1

[200 rows x 2 columns]

In [12]:
sample_df['class'].value_counts() 

1    101
0     99
Name: class, dtype: int64

In [13]:
def load_images(paths, img_size):
    images = []
    
    for path in paths:
        img = cv2.imread(path)
        img = cv2.resize(img, img_size) #resize the images
        img = np.array(img)
        
        images.append(img)
    
    images = np.array(images)
    images = images.astype(np.int64)
    
    #scale images
    images = images/255
    return images

In [14]:
images = load_images(sample_df['path'], (64, 64))
print(images.shape)

(200, 64, 64, 3)


In [15]:
x = images
y = sample_df["class"]

#train test split - 75% train, 25% test
x_training, x_test = x[:150,:], x[150:,:]
y_training, y_test = y[:150], y[150:]

In [16]:
x_training.shape

(150, 64, 64, 3)

In [17]:
x_test.shape

(50, 64, 64, 3)

In [18]:
y_training.shape

(150,)

In [19]:
y_test.shape

(50,)

In [20]:
# increases accuracy (even when not using loss function) 
tf.cast(x_training, tf.int64)
tf.cast(y_training, tf.int64)

2022-11-17 05:51:54.745061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-17 05:51:54.745133: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-17 05:51:54.745198: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-cyli6138): /proc/driver/nvidia/version does not exist
2022-11-17 05:51:54.745849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(150,), dtype=int64, numpy=
array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1])>

**4. Build the Model**

In [21]:
vehicle_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64, 3)),
    tf.keras.layers.Dense(16, activation='sigmoid', # first hidden layer has 16 inputs, each nodes activation will be a sigmoid function 
                          name='fc1', input_shape=(64, 64, 3)), 
    tf.keras.layers.Dense(3, name='fc2', activation='softmax')])

In [22]:
vehicle_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 fc1 (Dense)                 (None, 16)                196624    
                                                                 
 fc2 (Dense)                 (None, 3)                 51        
                                                                 
Total params: 196,675
Trainable params: 196,675
Non-trainable params: 0
_________________________________________________________________


**5. Train the Model & 6. Evaluate the Model**

In [23]:
vehicle_model.compile(optimizer='sgd', 
              loss="mse",
              metrics=['mae', 'mse', 'acc'])
vehicle_model.fit(x_training, y_training, 
                  epochs = 100, 
                  batch_size = 1,
                  verbose = 1)

Epoch 1/100
150/150 [==============================] - 1s 2ms/step - loss: 0.2953 - mae: 0.5156 - mse: 0.2953 - acc: 0.4733
Epoch 2/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2943 - mae: 0.5156 - mse: 0.2943 - acc: 0.4600
Epoch 3/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2940 - mae: 0.5156 - mse: 0.2940 - acc: 0.4533
Epoch 4/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2938 - mae: 0.5156 - mse: 0.2938 - acc: 0.4267
Epoch 5/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2937 - mae: 0.5156 - mse: 0.2937 - acc: 0.4067
Epoch 6/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2936 - mae: 0.5156 - mse: 0.2936 - acc: 0.3867
Epoch 7/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2935 - mae: 0.5156 - mse: 0.2935 - acc: 0.3733
Epoch 8/100
150/150 [==============================] - 0s 2ms/step - loss: 0.2935 - mae: 0.5156 - mse: 0.2935 - acc: 0.3533
Epoch 9/

**7. Make Predictions**

In [24]:
y_pred = vehicle_model.predict(x_test)
y_pred


2/2 [==============================] - 0s 3ms/step


array([[0.35240367, 0.31623143, 0.33136484],
       [0.34726664, 0.31010678, 0.3426266 ],
       [0.3293499 , 0.3508907 , 0.3197593 ],
       [0.33133316, 0.33629394, 0.33237284],
       [0.3317996 , 0.31992763, 0.3482727 ],
       [0.34148225, 0.31932512, 0.3391926 ],
       [0.33788165, 0.32083353, 0.34128478],
       [0.35115826, 0.31435674, 0.3344849 ],
       [0.33350545, 0.33678207, 0.3297124 ],
       [0.30976358, 0.34344846, 0.3467879 ],
       [0.3077073 , 0.38070872, 0.311584  ],
       [0.3444301 , 0.31841817, 0.33715162],
       [0.31009838, 0.34884477, 0.34105676],
       [0.32991993, 0.3423212 , 0.3277589 ],
       [0.34217373, 0.32816765, 0.32965863],
       [0.3449406 , 0.32060048, 0.33445898],
       [0.3436453 , 0.32759058, 0.32876417],
       [0.33134755, 0.33301574, 0.33563673],
       [0.33980158, 0.3250239 , 0.33517456],
       [0.339564  , 0.33070424, 0.32973185],
       [0.3369444 , 0.3211484 , 0.34190714],
       [0.337246  , 0.32782137, 0.3349326 ],
       [0.

In [25]:
test_loss = vehicle_model.evaluate(x,  y, verbose=2)

7/7 - 0s - loss: 0.2795 - mae: 0.5017 - mse: 0.2795 - acc: 0.3100 - 367ms/epoch - 52ms/step


In [26]:
probability_model = tf.keras.Sequential([vehicle_model, 
                                         tf.keras.layers.Softmax()])

In [27]:
predictions = probability_model.predict(x_test)

2/2 [==============================] - 0s 3ms/step


In [28]:
# Example: the first prediction. 
predictions[0]

array([0.3397137 , 0.32764506, 0.33264118], dtype=float32)

Predictions returns a number for each class which represents the probability of it being in that class. 